In [ ]:
!pip install flask-ngrok

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import torch.optim as opt
import torch.nn.functional as F

#CUDA_VISIBLE_DEVICES=""

class RegressionTrainDataset(Dataset):

    def __init__(self):
        # Inicijalizacija ili preuzimanje podataka  
        # Čitanje .csv fajla uz pomoć numpzy
        xy = np.loadtxt('regression_train.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.br_uzoraka = xy.shape[0]
      
        #prva kolona je ciljana vrednost, ostalo su svojstva
        self.x_data = torch.from_numpy(xy[:, 1:]) # [br_uzoraka, br_svojstava]
        self.y_data = torch.from_numpy(xy[:, [0]]) # [br_uzoraka, 1]

    # dataset[i] vraća i-ti uzorak
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # pozivom len(dataset) vraća veličinu skupa podataka
    def __len__(self):
        return self.br_uzoraka

# kreirati objekat zapredstavljanje skupa podataka obuke
train_set = RegressionTrainDataset()


class RegressionTestDataset(Dataset):

    def __init__(self):
        # Inicijalizacija ili preuzimanje podataka  
        # Čitanje .csv fajla uz pomoć numpzy
        xy = np.loadtxt('regression_test.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.br_uzoraka = xy.shape[0]
      
        #prva kolona je ciljana vrednost, ostalo su svojstva
        self.x_data = torch.from_numpy(xy[:, 1:]) # [br_uzoraka, br_svojstava]
        self.y_data = torch.from_numpy(xy[:, [0]]) # [br_uzoraka, 1]

    # dataset[i] vraća i-ti uzorak
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # pozivom len(dataset) vraća veličinu skupa podataka
    def __len__(self):
        return self.br_uzoraka


# Čitamo ceo skup podataka uz pomoč DataLoader
# shuffle: promešati podatke - dobro za obuku 
# num_workers: brže se učitava za više procesora
train_loader = DataLoader(dataset=train_set,
                          batch_size=5,
                          shuffle=True,
                          num_workers=2)


class MyNeuralNetwork(torch.nn.Module):
    def __init__(self, input_size):
        super(MyNeuralNetwork, self).__init__()

        self.input_size = input_size

        self.linear_1 = torch.nn.Linear(input_size, 20)
        self.linear_2 = torch.nn.Linear(20, 10)
        self.linear_3 = torch.nn.Linear(10, 1)

    def forward(self, x):
        out = F.relu(self.linear_1(x))
        out = F.relu(self.linear_2(out))
        out = self.linear_3(out)
        return out

mreza=MyNeuralNetwork(3)

optimizer=opt.Adam(mreza.parameters(), lr=0.01)
f_greske=torch.nn.MSELoss()

# Petlja za obuku
br_epoha = 15
ukupno_uzoraka = len(train_set)

for epoch in range(br_epoha):
    for i, (ulazi, izlazi) in enumerate(train_loader):
        predikcije=mreza(ulazi)
        gubici=f_greske(predikcije,izlazi)
       # print('Ocekivano', izlazi)
       # print('Predikcija', predikcije)
        gubici.backward()
        optimizer.step()
        mreza.zero_grad()

test_set = RegressionTestDataset()
test_loader = DataLoader(dataset=test_set,
                          batch_size=1,
                          shuffle=True,
                          num_workers=1)
ukupno_test=len(test_set)
print('Ukupno test', ukupno_test)

for i, (ulazi, izlazi) in enumerate(test_loader):
  predikcije=mreza(ulazi)
  print('Ulazi',ulazi)
  print('Ocekivano', izlazi)
  print('Predikcija', predikcije)
  print('Greska', abs(predikcije-izlazi))



from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import request
  
app = Flask(__name__)
run_with_ngrok(app)
  

@app.route("/predict")
def drugi():
    osobe = float(request.args.get('osobe'))
    sezona = float(request.args.get('sezona'))
    soba = float(request.args.get('soba'))

    ulazi=torch.tensor([[osobe, sezona, soba]])
    predikcije=mreza(ulazi)
    odgovor=str(predikcije[0].item())
    return odgovor
  
if __name__ == "__main__":
  app.run()

Ukupno test 7
Ulazi tensor([[3., 1., 1.]])
Ocekivano tensor([[300.]])
Predikcija tensor([[19.1130]], grad_fn=<AddmmBackward>)
Greska tensor([[280.8870]], grad_fn=<AbsBackward>)
Ulazi tensor([[1., 0., 1.]])
Ocekivano tensor([[50.]])
Predikcija tensor([[9.7432]], grad_fn=<AddmmBackward>)
Greska tensor([[40.2568]], grad_fn=<AbsBackward>)
Ulazi tensor([[3., 1., 2.]])
Ocekivano tensor([[400.]])
Predikcija tensor([[23.3597]], grad_fn=<AddmmBackward>)
Greska tensor([[376.6403]], grad_fn=<AbsBackward>)
Ulazi tensor([[1., 1., 3.]])
Ocekivano tensor([[600.]])
Predikcija tensor([[21.1716]], grad_fn=<AddmmBackward>)
Greska tensor([[578.8284]], grad_fn=<AbsBackward>)
Ulazi tensor([[3., 0., 2.]])
Ocekivano tensor([[300.]])
Predikcija tensor([[20.4246]], grad_fn=<AddmmBackward>)
Greska tensor([[279.5754]], grad_fn=<AbsBackward>)
Ulazi tensor([[3., 0., 1.]])
Ocekivano tensor([[200.]])
Predikcija tensor([[16.1779]], grad_fn=<AddmmBackward>)
Greska tensor([[183.8221]], grad_fn=<AbsBackward>)
Ulazi tenso

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://106d-35-245-186-64.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [03/Sep/2021 13:25:10] "GET /predict?osobe=2&sezona=1&soba=2 HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2021 13:25:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Sep/2021 13:25:22] "GET /predict?osobe=2&sezona=1&soba=3 HTTP/1.1" 200 -
127.0.0.1 - - [03/Sep/2021 13:25:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Sep/2021 13:25:41] "GET /predict?osobe=5&sezona=1&soba=3 HTTP/1.1" 200 -
